## Introduction

Copywriting AI assistants like Jasper and CopyAI are great, but they're missing one key ingredient: knowledge of your specific business. Let's fix that.

In this tutorial we're going to show you how to build a simple copywriting assistant that can leverage information about your business in order to write highly relevant blog posts.

## Prerequisites

Before we start, ensure you have the following:

1. A Superpowered AI account with API keys (Sign up [here](https://superpowered.ai) for free access).
2. The Superpowered AI Python SDK installed. You can install it using pip if you don't have it already: `pip install superpowered-sdk`
3. An OpenAI account with an API key created, and the openai Python SDK

## Step-by-Step Tutorial

### Step 1: Set up the environment

First, we will set up the environment by importing the required libraries and initializing the Superpowered AI SDK with our API keys.

In [1]:
from superpowered import create_knowledge_base, query_knowledge_bases, create_document_via_url, create_document_via_file
import openai
import os

In [ ]:
# set API keys
os.environ["SUPERPOWERED_API_KEY_ID"] = "YOUR_API_KEY_ID"
os.environ["SUPERPOWERED_API_KEY_SECRET"] = "YOUR_API_KEY_SECRET"
os.environ["OPENAI_API_KEY"] = "YOUR_API_KEY"

Replace `"YOUR_API_KEY_ID"` and `"YOUR_API_KEY_SECRET"` with your actual API keys.

### Step 2: Create a Knowledge Base

In [5]:
# create knowledge base and upload some files
kb = create_knowledge_base(title="Superpowered Documentation")
knowledge_base_id = kb["id"]

Alright now we need to upload all the content we want in our Knowledge Base. I'm going to go ahead and copy and paste everything from our Notion docs into a txt file, and then upload that.

In [6]:
file_path = "/Users/zach/Code/superpowered_notion_docs.txt"
create_document_via_file(knowledge_base_id=knowledge_base_id, file_path=file_path)

{'message': 'Successfully uploaded file. Vectorization process will begin shortly.'}

And then I'm going to find a few relevant articles about complementary and competitive products that may be helpful for the AI to know about. 

In [7]:
create_document_via_url(knowledge_base_id=knowledge_base_id, url="https://towardsdatascience.com/milvus-pinecone-vespa-weaviate-vald-gsi-what-unites-these-buzz-words-and-what-makes-each-9c65a3bd0696")

{'message': 'Url submitted for scraping. Please use the GET /knowledge_bases/{kb_id}/documents to check the status of the document.'}

### Step 3: Create our Copywriting Assistant using GPT-3

Now that we have our Knowledge Base looking good, we can start building the copywriting assistant. We'll start by defining the prompt template. 

In [8]:
template = """
You are an AI writing assistant. You have been given the task of writing a blog post with the following premise:
{premise}

And here is some information from the company's knowledge base that you may find relevant to the task at hand:
{relevant_info}
""".strip()

Now we can define our variables and make a GPT-3 call to actually write the blog post! Before making the GPT-3 API call, we'll first make an API call to our Superpowered AI Knowledge Base to retrieve information relevant to the premise of the blog post. We can use either the raw results, or the LLM-generated summary. The raw results can look a little messy, but they usually perform better than just using the summary, since there's no information loss.

In [ ]:
premise = "Write a blog post about all the possible use cases of Superpowered AI. You don't have to go into much detail about each one. Write in a passionate and excited voice."

# query top 10 results from the knowledge base we just created
kb_query_results = query_knowledge_bases(knowledge_base_ids=[knowledge_base_id], query=premise, top_k=10, summarize_results=True)

# print the results
#relevant_info = kb_query_results["summary"] # if you just want to use the summary
relevant_info = "\n\n".join([kb_query_results["ranked_results"][i]["content"] for i in range(len(kb_query_results["ranked_results"]))]) # use raw results
print (relevant_info)

Now we format our prompt with the premise and the relevant info we retrieved from our Knowledge Base.

In [ ]:
prompt = template.format(premise=premise, relevant_info=relevant_info)
print (prompt)

And now we actually make the OpenAI API call. We'll use GPT-3.5-Turbo since it provides a great combination of speed and performance, but you could also use GPT-4 if you want higher quality.

In [11]:
openai.api_key = os.getenv("OPENAI_API_KEY")
user_input_message = {"role": "user", "content": prompt}
messages = [user_input_message]
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=messages,
    max_tokens=1500,
    temperature=0.7,
    request_timeout=120,
)
llm_output = response['choices'][0]['message']['content'].strip()

print (llm_output)

Are you ready to unleash the full potential of AI? Look no further than Superpowered AI! With its Knowledge Base as a Service (KBaaS), this managed solution is your ticket to improved query performance and user experience. The possibilities are endless with Superpowered AI, from autonomous agent web browsing to dense vector search. And the best part? You don't have to be an expert in AI technology to use it. 

Creating and querying from Knowledge Bases has never been easier thanks to Superpowered AI's Playground, API, and Python SDK. And for a limited time, Superpowered AI is offering free access to its services. 

So why wait? Join the AI revolution with Superpowered AI and take your projects to the next level. The future is here, and it's Superpowered!


Not bad. If you actually want to build a copywriting assistant that competes with Jasper and CopyAI, you'll need to build something quite a bit more sophisticated than this. But the basic idea of inserting the retrieved results from the Superpowered AI Knowledge Base into the prompt will remain the same.